# Mini-Project - Develop a "Problem Solving Walkthrough" 

In this mini-project, you I wrote a "problem solving walkthrough". I chose a programming problem, and solved it by documenting the process.

## Walkthrough
The problem solving **walkthrough** is a written **guided description** of the journey from a problem to a solution. Detailed reasoning is performed and written for each one of the phases. For example, in the first phase, besides stating the problem in your own words you should include at least a few input-output pairs. As another example, a well-performed design phase contains several drafts showing gradual improvements. It should also justify your choice of datastructures.

**Incremental development:** Get something working and keep it working. 


### Credits:
    
The puzzle has been created by Eric Wastl as part of the Advent of Code project.

[Advent of Code Problem: 2018 - Day 7](https://adventofcode.com/2018/day/7)

In [56]:
import os

assert os.path.exists('data/input.txt')

## The Sum of Its Parts

The instructions from the input file specify a series of steps and requirements about which steps must be finished before others can begin. Each step is designated by a single letter. For example, suppose you have the following instructions:

    Step C must be finished before step A can begin.  
    Step C must be finished before step F can begin.  
    Step A must be finished before step B can begin.  
    Step A must be finished before step D can begin.  
    Step B must be finished before step E can begin.  
    Step D must be finished before step E can begin.  
    Step F must be finished before step E can begin.  

Visually, these requirements look like this:


      -->A--->B--
     /    \      \
    C      -->D----->E
     \           /
      ---->F-----

Your first goal is to determine the order in which the steps should be completed. If more than one step is ready, choose the step which is first alphabetically. In this example, the steps would be completed as follows:

    Only C is available, and so it is done first.  
    Next, both A and F are available. A is first alphabetically, so it is done next.  
    Then, even though F was available earlier, steps B and D are now also available, and B is the first alphabetically of the three.  
    After that, only D and F are available. E is not available because only some of its prerequisites are complete. Therefore, D is completed next.  
    F is the only choice, so it is done next.  
    Finally, E is completed.  

So, in this example, the correct order is CABDFE.

**Expected Output:** 

In what order should the steps in your instructions be completed?

## And Now, Let's Solve It!

<div class="alert alert-info">
  <h4>Programming Problem Solving Model</h4>
  <ol>
    <li>Reinterpret the Problem</li>
    <li>Design a Solution</li>
    <li>Code</li>
    <li>Test</li>
    <li>Debug</li>
    <li>Evaluate & Reflect</li>
  </ol>
</div>

<div class="alert alert-info">
  <h4>Incremental Development</h4>
  <ul>
    <li>Rapid cycles of Problem + Design + Code + Test + Debug</li>
    <li>Start small and keep it working</li>
  </ul>
</div>

<a id="problem-phase-top"></a>
<div class="alert alert-info">
<h3>Reinterpret the Problem</h3>
</div>

A set of instructions is available.   
Each instruction has a form:   
Step X must be finished before step Y can begin.

Goal: find the correct order of steps.  
Desired output form: a string with letters representing the correct order. 

If more than one step can be taken at one point, take the step which is alphabetically first.

-----

Examples of input-output pairs: 

### Example 1

Input:   
Step A must be finished before step C can begin.      
Step A must be finished before step D can begin.     
Step C must be finished before step B can begin.     
Step D must be finished before step B can begin.   


      -->C
     /    \      
    A        -->B
     \      /
      ---->D


Output:   
"ACDB" 

### Example 2

Input:   
Step C must be finished before step D can begin.  
Step D must be finished before step A can begin.  
Step D must be finished before step E can begin.  


           -->A
          /    \      
    C -->D  ----->E
 
 
 Output: "CDAE"

 

<div class="alert alert-info">
<h3>Design a Solution </h3>
</div>

## Draft One

**What do we need to create the output string of correct order of steps?**

1. An alphabetically ordered set of available steps.   
2. A starting point and a condition to terminate.
3. Per iteration:   
    - First step from the available steps set added to the string of correct order. 
    - Step above is removed from the set of available steps.  
    - Steps that depend only step above are added to the set of available steps.  


**1. What do we need to build the available steps set?**
- Look-up possibility: given a step, which dependencies does it have? 
- If a step doesn't have any dependencies: add it to the set of available steps. 

**2. Starting point and program termination**
- Add steps that have do not have any dependencies to the set of available steps.
- Remove the step above from the dictionary of dependencies
- When dependencies dictionary is empty: program terminates. 
    - This procedure does assume that all steps have to be taken and that the input file does not contain infinate loops
    
**3. Iterative Step**
- Add the first step from the alphabetically sorted list of available steps 
    - Note: The set of available steps is converted into a list upon sorting
- Remove it from the available steps
- Add the steps that depend on it to the set of available steps
    - Note: transform the sorted list of available steps back to a set
- If no steps were added: iterative step is repeated anyway


### Choice and Reasoning behind Data Structures: 

_dependencies_:  
Dictionary.     
A dictionary provides an easy access to a list of newly available steps after a step is finished.  
Key = step  
Value = set of steps that depend on it. Quick and easy look-up. No need for order, no repetitions allowed.

_Available steps_: initially a set. Cheap look-up, no repetitions.   
Turned into a list each time _sorted()_ function is applied to it. 

_Steps in order_: a string to which steps will be appended. Could be a list, doesn't matter much.

### Breaking into Subproblems
So, we have **subproblems**: 
1. Create a look-up possibility for: list what dependencies does a step have
2. Starting point: list steps, which dependencies do not appear as a key in the dictionary, meaning that they have no dependencies.
3. Iteratively add the steps to _available steps_ set and take them in alphabetical order.

<div class="alert alert-info">
<h3>Code</h3>
</div>

In [57]:
from collections import defaultdict

## Code for First Subproblem: Creating Look-up Dictionary

In [58]:
#solve first subproblem
def create_dependencies_dictionary(filename):
    '''Creates a dictionary with steps as keys and a list of required previous steps as values
    
    :param filename: filename with step descriptions. One step description per line.
    :return: a dictionary of dependencies'''
    dependencies = defaultdict(list)

    with open(filename, 'r') as instructions_file:
        for line in instructions_file:
            if len(line) > 1:
                step_to_begin = line[-13]
                dependency_step = line[5]

                dependencies[step_to_begin].append(dependency_step)
                
    return dependencies

In [59]:
#solve second subproblem
def find_possible_starts(available_steps, dependencies):
    '''Finds possible starts: sets that do not require any other previous steps. Adds them to
    available steps set

    :param available_steps: empty list where possible starts will be stored
    :param dependencies: dict with keys: steps, values=set of their dependencies
    :return: list of possible starts '''


    for step, dep_steps in sorted(dependencies.items()):
        for dep_step in dep_steps:
            if dep_step not in dependencies:
                available_steps.add(dep_step)

    print("Avaliable: ", available_steps)   
    return available_steps

In [60]:
#solve third subroblem
def take_step(available_steps, steps_in_order, dependencies):
    '''
    Takes the first element from the sorted list of available steps to ensure alphabetical
    taking of available steps. Adds it to string of steps in order.
    Removes the step taken from dependencies.
    
    :param available_steps: set of available steps
    :param steps_in_order: string with current steps in order
    :param dependencies: dict with keys: steps, values=set of their dependencies'''
    
    step_to_take = sorted(available_steps)[0]
    steps_in_order += step_to_take
    available_steps.remove(step_to_take)
    if step_to_take in dependencies:
        del dependencies[step_to_take]
        
    return step_to_take, steps_in_order

In [61]:
def add_available_steps(step_taken, available_steps, dependencies):
    ''' Adds steps that became available after a step has been taken to the set
    of available steps.
    Removes a taken steps from sets of depedendencies in dependencies dict'''
    
    for step, dep_steps in dependencies.items():
        if step_taken in dep_steps:
            dep_steps.remove(step_taken)
        if dep_steps == []:
            available_steps.add(step)
        
    return available_steps

In [62]:
def main(filename):
    dependencies = create_dependencies_dictionary(filename)
    
    #display dictionary with dependencies
    print("Dictionary of Dependencies: ")
    for key, value in sorted(dependencies.items()):
        print(key, ": ", value)
        
    available_steps = set()
    find_possible_starts(available_steps, dependencies)
    
    steps_in_order = ""
    
    while dependencies.keys():
        
        step_taken, steps_in_order = take_step(available_steps, steps_in_order, dependencies)
        print("Step taken: ", step_taken[0])
        print("Steps in Order", steps_in_order)


        add_available_steps(step_taken, available_steps, dependencies)
        print("Updated Available Steps: ", available_steps)

    
    return steps_in_order


<div class="alert alert-info">
<h3>Test (3 pts)</h3>
</div>

In [63]:
main('data/input.txt')


Dictionary of Dependencies: 
A :  ['W', 'L', 'Y', 'J', 'U', 'K']
C :  ['J', 'H', 'Y', 'W', 'A', 'L', 'I', 'P']
D :  ['O', 'J', 'P', 'H', 'N', 'Q', 'C', 'A']
E :  ['J', 'A', 'C', 'D', 'S', 'O', 'Q', 'G', 'I']
H :  ['X', 'N', 'I']
I :  ['F', 'X', 'T']
J :  ['U', 'Z', 'Y', 'M']
K :  ['B']
L :  ['K', 'G', 'M', 'P']
M :  ['K', 'U', 'R']
N :  ['F']
O :  ['U', 'Z', 'L', 'P', 'Q', 'V', 'K', 'N']
P :  ['H', 'I', 'G']
Q :  ['H', 'U', 'W', 'T']
R :  ['B']
S :  ['N', 'C', 'O', 'X', 'Z', 'Y', 'Q', 'L', 'D']
T :  ['B']
U :  ['T', 'I', 'F']
V :  ['H', 'W', 'T', 'P', 'L']
W :  ['T', 'R']
X :  ['R']
Y :  ['X', 'I', 'V', 'Q', 'P', 'O', 'R', 'N', 'B', 'W']
Z :  ['R', 'T', 'G', 'B']
Avaliable:  {'B', 'G', 'F'}
Step taken:  B
Steps in Order B
Updated Available Steps:  {'K', 'F', 'G', 'R', 'T'}
Step taken:  F
Steps in Order BF
Updated Available Steps:  {'K', 'N', 'G', 'R', 'T'}
Step taken:  G
Steps in Order BFG
Updated Available Steps:  {'K', 'N', 'R', 'T'}
Step taken:  K
Steps in Order BFGK
Updated Availab

'BFGKNRTWXIHPUMLQVZOYJACDSE'

### Improvement Needed: Efficiency 
The code works, but there is a way to make it more efficient: 
    create a second look-up dictionary with key = step, values = steps that depend on that key.
Then, instead of looping over entire dependencies each time a step has been taken to find which
steps required it as their dependencies,
we just loop over steps that depend on the step taken.

For example,

        if steps A,C, and D depend on B:
            the entry in the 
            {'B' : ['A', 'C', 'D']}
            
Once B is accomplished: we remove it from the sets of depencencies of three steps A,C, and D.

_dependencies_ dictionary: key=step, value=its dependencies  
_steps that follow_ dictionary: key=step, value=steps that follow the key
    

In [64]:
def create_dependencies_dictionaries(filename):
    '''Creates a dictionary with steps as keys and a list of required previous steps as values
    
    :param filename: filename with step descriptions. One step description per line.
    :return: a dictionary of dependencies'''
    
    dependencies = defaultdict(set)
    steps_that_follow = defaultdict(set)

    with open(filename, 'r') as instructions_file:
        for line in instructions_file:
            if len(line) > 1:
                step_to_begin = line[-13]
                dependency_step = line[5]

                dependencies[step_to_begin].add(dependency_step)
                steps_that_follow[dependency_step].add(step_to_begin)
                
    return dependencies, steps_that_follow

In [65]:
def find_possible_starts2(dependencies):
    '''Finds possible starts: stes that do not require any other previous steps.
    And remove 

    :param available_steps: empty list where possible starts will be stored
    :return: list of possible starts '''

    available_steps = set()
    
    for step, dep_steps in dependencies.items():
        for dep_step in dep_steps:
            if dep_step not in dependencies:
                available_steps.add(dep_step)

    return available_steps

In [66]:
def take_step2(available_steps, steps_in_order, dependencies):
    step_to_take = sorted(available_steps)[0]
    steps_in_order += step_to_take
    available_steps.remove(step_to_take)
    if step_to_take in dependencies:
        del dependencies[step_to_take]
        
    return step_to_take, steps_in_order

In [67]:
def add_available_steps2(step_taken, available_steps, dependencies, steps_that_follow):
    ''' Adds steps that became available after a step has been taken to the set
    of available steps.
    Removes a taken steps from sets of depedendencies in dependencies dict'''
    
    
    for step in steps_that_follow[step_taken]:
        dependencies[step].remove(step_taken)
        if dependencies[step] == set():
            available_steps.add(step)
        
    return available_steps

In [68]:
def main(filename):
    
    dependencies, steps_that_follow = create_dependencies_dictionaries(filename)
    available_steps = find_possible_starts2(dependencies)
    
    steps_in_order = ""
    
    while dependencies.keys():
        
        step_taken, steps_in_order = take_step2(available_steps, steps_in_order, dependencies)
        print("Step taken: ", step_taken)
        print("Steps in Order", steps_in_order)

        add_available_steps2(step_taken, available_steps, dependencies, steps_that_follow)
        
    return steps_in_order
    

## Test

In [22]:
assert main('data/input_test01.txt') == 'CABDFE'

Step taken:  C
Steps in Order C
Step taken:  A
Steps in Order CA
Step taken:  B
Steps in Order CAB
Step taken:  D
Steps in Order CABD
Step taken:  F
Steps in Order CABDF
Step taken:  E
Steps in Order CABDFE


In [69]:
assert main('data/input_test02.txt') == 'ACDB'

Step taken:  A
Steps in Order A
Step taken:  C
Steps in Order AC
Step taken:  D
Steps in Order ACD
Step taken:  B
Steps in Order ACDB


In [70]:
assert main('data/input_test03.txt') == 'CDAE'

Step taken:  C
Steps in Order C
Step taken:  D
Steps in Order CD
Step taken:  A
Steps in Order CDA
Step taken:  E
Steps in Order CDAE


In [71]:
assert main('data/input_test04.txt') == 'AFCDGBKHI'

Step taken:  A
Steps in Order A
Step taken:  F
Steps in Order AF
Step taken:  C
Steps in Order AFC
Step taken:  D
Steps in Order AFCD
Step taken:  G
Steps in Order AFCDG
Step taken:  B
Steps in Order AFCDGB
Step taken:  K
Steps in Order AFCDGBK
Step taken:  H
Steps in Order AFCDGBKH
Step taken:  I
Steps in Order AFCDGBKHI


In [72]:
main('data/input.txt')

Step taken:  B
Steps in Order B
Step taken:  F
Steps in Order BF
Step taken:  G
Steps in Order BFG
Step taken:  K
Steps in Order BFGK
Step taken:  N
Steps in Order BFGKN
Step taken:  R
Steps in Order BFGKNR
Step taken:  T
Steps in Order BFGKNRT
Step taken:  W
Steps in Order BFGKNRTW
Step taken:  X
Steps in Order BFGKNRTWX
Step taken:  I
Steps in Order BFGKNRTWXI
Step taken:  H
Steps in Order BFGKNRTWXIH
Step taken:  P
Steps in Order BFGKNRTWXIHP
Step taken:  U
Steps in Order BFGKNRTWXIHPU
Step taken:  M
Steps in Order BFGKNRTWXIHPUM
Step taken:  L
Steps in Order BFGKNRTWXIHPUML
Step taken:  Q
Steps in Order BFGKNRTWXIHPUMLQ
Step taken:  V
Steps in Order BFGKNRTWXIHPUMLQV
Step taken:  Z
Steps in Order BFGKNRTWXIHPUMLQVZ
Step taken:  O
Steps in Order BFGKNRTWXIHPUMLQVZO
Step taken:  Y
Steps in Order BFGKNRTWXIHPUMLQVZOY
Step taken:  J
Steps in Order BFGKNRTWXIHPUMLQVZOYJ
Step taken:  A
Steps in Order BFGKNRTWXIHPUMLQVZOYJA
Step taken:  C
Steps in Order BFGKNRTWXIHPUMLQVZOYJAC
Step taken:

'BFGKNRTWXIHPUMLQVZOYJACDSE'

## --- Part Two ---

Now, you need to account for multiple people working on steps simultaneously. If multiple steps are available, workers should still begin them in alphabetical order.

Each step takes 60 seconds plus an amount corresponding to its letter: A=1, B=2, C=3, and so on. So, step A takes 60+1=61 seconds, while step Z takes 60+26=86 seconds. No time is required between steps.

To simplify things for the example, however, suppose you only have help from one Elf (a total of two workers) and that each step takes 60 fewer seconds (so that step A takes 1 second and step Z takes 26 seconds). Then, using the same instructions as above, this is how each second would be spent:

    Second   Worker 1   Worker 2   Done
       0        C          .        
       1        C          .        
       2        C          .        
       3        A          F       C
       4        B          F       CA
       5        B          F       CA
       6        D          F       CAB
       7        D          F       CAB
       8        D          F       CAB
       9        D          .       CABF
      10        E          .       CABFD
      11        E          .       CABFD
      12        E          .       CABFD
      13        E          .       CABFD
      14        E          .       CABFD
      15        .          .       CABFDE
      
      
Each row represents one second of time. The Second column identifies how many seconds have passed as of the beginning of that second. Each worker column shows the step that worker is currently doing (or . if they are idle). The Done column shows completed steps.

Note that the order of the steps has changed; this is because steps now take time to finish and multiple workers can begin multiple steps simultaneously.

In this example, it would take 15 seconds for two workers to complete these steps.

**With 5 workers and the 60+ second step durations described above, how long will it take to complete all of the steps?**

## Design  


### Subtask one:  
create a dictionary with key=step, value=its ducration  

Design:  
- create a list containing all steps, which are all uppercase letters of the English alphabet
- use enumerate function starting from 60 to create durations for each step  
- create a dictionary with key=step, value=duration  

### Subtask Two

Keep track of the following:
- if worker is free or busy
- which tasks become available
- if all workers are busy, the next task should be allocated to a worker that will be free the soonest  


**Data structures**

Dictionary for alphabet and ducration dicts: for ease of look-up

Set of available steps (same reasoning as in part one)

Lists for 
1. is worker busy
2. how much time does every worker need to complete theor task
3. which task they are working on

Why lists?
Position in lists reflect the index of worker.

        is_worker_busy = [False, False, False, False, False]
        time_elapsed_for_worker = [0, 0, 0, 0, 0]
        working_on_task = ['', '', '', '', '']



In [74]:
def create_alphabet_dict():
    '''Creates two dictionaries:
    - Letter: its duration 
    and
    - Duration : its letter'''
    
    alphabet_list = [chr(i) for i in range(ord('A'),ord('Z')+1)]
    alphabet_seconds = list(enumerate(alphabet_list, 61))
    alphabet_dict = {letter:seconds for (seconds,letter) in alphabet_seconds}
    duration_dict = {str(seconds):letter for (seconds,letter) in alphabet_seconds}
    
    return alphabet_dict, duration_dict

In [75]:
def take_step(available_steps, dependencies):

    step_to_take = sorted(available_steps)[0]
    available_steps.remove(step_to_take)
    if step_to_take in dependencies:
        del dependencies[step_to_take]
    
        
    return step_to_take

In [76]:
def add_available_steps(step_taken, available_steps, dependencies, steps_that_follow):
    
    for step in steps_that_follow[step_taken]:
        if step in dependencies and step_taken in dependencies[step]:
            dependencies[step].remove(step_taken)
            if dependencies[step] == set():
                available_steps.add(step)
        
    return available_steps

In [77]:
def main(filename):    
    
    #Subtask One
    alphabet_dict, duration_dict = create_alphabet_dict()
    dependencies, steps_that_follow = create_dependencies_dictionaries(filename)
    available_steps = find_possible_starts2(dependencies)
    
    #Subtask two
    time_elapsed = 0
    is_worker_busy = [False, False, False, False, False]
    time_elapsed_for_worker = [0, 0, 0, 0, 0]
    working_on_task = ['', '', '', '', '']

    for i in range(40):
        if not available_steps:
            time_elapsed_for_worker_no_zeros = [num for num in time_elapsed_for_worker 
                                                if num != 0]
            step_quickest_to_finish = duration_dict[str(min(time_elapsed_for_worker_no_zeros))]
            add_available_steps(step_quickest_to_finish, available_steps, 
                                 dependencies, steps_that_follow)
            working_on_task[working_on_task.index(step_quickest_to_finish)] = ""
                
                
                
        elif available_steps:
            #find lowest time elapsed
            next_available_time_slot = min(time_elapsed_for_worker)
            #find index of worker with least time elapsed
            next_available_worker_index = time_elapsed_for_worker.index(next_available_time_slot)
            is_worker_busy[next_available_worker_index] = True
            step_taken = sorted(available_steps)[0]
            working_on_task[next_available_worker_index] = step_taken
            time_elapsed_for_worker[next_available_worker_index] += alphabet_dict[step_taken]
            time_elapsed += alphabet_dict[step_taken]
            step_taken = take_step(available_steps, dependencies)

            print('Time Elapsed:', time_elapsed)
            print('is_worker_busy:', is_worker_busy)
            print('time_elapsed_for_worker', time_elapsed_for_worker)
            print('working_on_task', working_on_task)
            print("Step taken: ", step_taken)
            print('available_steps', available_steps)
    


In [78]:
main('data/input.txt')

Time Elapsed: 62
is_worker_busy: [True, False, False, False, False]
time_elapsed_for_worker [62, 0, 0, 0, 0]
working_on_task ['B', '', '', '', '']
Step taken:  B
available_steps {'G', 'F'}
Time Elapsed: 128
is_worker_busy: [True, True, False, False, False]
time_elapsed_for_worker [62, 66, 0, 0, 0]
working_on_task ['B', 'F', '', '', '']
Step taken:  F
available_steps {'G'}
Time Elapsed: 195
is_worker_busy: [True, True, True, False, False]
time_elapsed_for_worker [62, 66, 67, 0, 0]
working_on_task ['B', 'F', 'G', '', '']
Step taken:  G
available_steps set()
Time Elapsed: 266
is_worker_busy: [True, True, True, True, False]
time_elapsed_for_worker [62, 66, 67, 71, 0]
working_on_task ['', 'F', 'G', 'K', '']
Step taken:  K
available_steps {'R', 'T'}
Time Elapsed: 344
is_worker_busy: [True, True, True, True, True]
time_elapsed_for_worker [62, 66, 67, 71, 78]
working_on_task ['', 'F', 'G', 'K', 'R']
Step taken:  R
available_steps {'T'}
Time Elapsed: 424
is_worker_busy: [True, True, True, True,

ValueError: 'G' is not in list

## Debug

<div class="alert alert-warning">
<h3>Debug</h3>

1. Reproduce
2. Diagnose
3. Fix
4. Reflect

</div>

##### 1. Reproduce

The Error Message is the following

    ValueError: 'G' is not in list.  
    
The problem occures in the line

       working_on_task[working_on_task.index(step_quickest_to_finish)] = ""

##### 2. Diagnose
A step is not in the working_on_task list.    
This happends because we try to remove a step from the list of tasks that workers are currently working on after adding the steps made available after acomplishing this task.    
E.g.: 'G' is the quickest to acomplish. Remove 'G' as dependency from all steps that rquire it. Add steps that now don't have any dependencies to available_steps list.    

The issue is: G is just removed, no new steps are added.  

##### 3. Fix
We need to simulate a timeline. 
First assign all available steps to all available workers. 

If more workers than steps available: the remaining workers can only start their tasks after the task with quickest duration is finished and maybe some new steps are made available through this.

When all workers busy: 
the worker that will be do the soonest finishes, and new tasks may be made available. If now: next worker finishes, check if now tasks are available. Continue until find available tasks.

**Data Structures**


1. list of workers. Element of list set to True if the worker has a task, False if free  
2.2. 

Why list?
Position of a list simulates the number of a worker: 0 for 1st worker, 4 for 5th worker.

2. Priority Queue with available steps

Data structure: Sorted list. Initially available_steps is a set for ease of adding new elements and removing them.

    sorted(available_steps)[0]

3. Priority queue with seconds elapsed for each worker. Once every worker started a task, tasks will be allocate to workers that finish their steps the quickest.

         sorted(priority_queue_time)[0]

##### 4. Reflect
Simulating a timeline was a challange. I had to make sure to separate the tasks that have been started and the tasks that are finished. 

Thinking about this task in form of a priority queue enabled me to simulate a timeline.


In [79]:
def take_step(available_steps, is_worker_busy, alphabet_dict, dependencies, priority_queue_time):
    
    '''
    Start taking step. Assign it to a worker. Remove it from available steps and
    dependencies dictionary.

    '''
    
    #for the beginning
    step_to_take = sorted(available_steps)[0]
    duration_of_step = alphabet_dict[step_to_take]
    free_worker = is_worker_busy.index(False)

    #remove step from available 
    available_steps.remove(step_to_take)

    #set worker to busy (True)
    is_worker_busy[free_worker] = True

    #delete from dependencies
    if step_to_take in dependencies:
        del dependencies[step_to_take]
        
    return duration_of_step, step_to_take, free_worker

In [80]:
def add_available_steps(finished_step, available_steps, dependencies, steps_that_follow):
    
    '''Add steps made available after a step has been finished'''
    
    if finished_step in steps_that_follow:
        for step in steps_that_follow[finished_step]:
            if finished_step in dependencies[step]:
                dependencies[step].remove(finished_step)
                if dependencies[step] == set():
                    del dependencies[step]
                    available_steps.add(step)
        return available_steps
    
    else:
        return False


In [93]:
def main(filename):
    
    alphabet_dict, duration_dict = create_alphabet_dict()
    dependencies, steps_that_follow = create_dependencies_dictionaries(filename)
    available_steps = find_possible_starts2(dependencies)
    
    steps_in_order = []
    time_elapsed = 0
    is_worker_busy = [False, False, False, False, False]

    priority_queue_time = []
    
    while dependencies.keys() or available_steps:
        if available_steps and False in is_worker_busy:
            print('available steps', available_steps)
            step_to_append = take_step(available_steps, is_worker_busy,
                                       alphabet_dict, dependencies, priority_queue_time)
            print("Step to append: ", step_to_append)
            
            search = step_to_append[2]
            if any(step_in_queue[2] == search for step_in_queue in priority_queue_time):
                step_to_replace = list(filter(lambda x:x[2]==search, priority_queue_time))
                print('step to replace', step_to_replace)
                updated_time = step_to_replace[0][0] + step_to_append[0]
                step_to_append = (updated_time, step_to_append[1], step_to_append[2])
                priority_queue_time.remove(step_to_replace[0])
                priority_queue_time.append(step_to_append)

            else:
                updated_time = time_elapsed + step_to_append[0]
                step_to_append = (updated_time, step_to_append[1], step_to_append[2])
                priority_queue_time.append(step_to_append)
                     
            print("priority queue with appended step:", priority_queue_time)
            
            print('after taking step is worker busy', is_worker_busy)
            
            

            
        elif not available_steps and dependencies.keys():
    
            #add newly available steps 
            counter = 0
            while not available_steps and counter <= len(priority_queue_time):
                print('counter for available steps:', counter)
                quickest_finished_step = sorted(priority_queue_time)[0]
                time_elapsed = sorted(priority_queue_time)[0][0]

                #set worker to free for the task that will be done the soonest
                is_worker_busy[sorted(priority_queue_time)[0][2]] = False

                available_steps = add_available_steps(quickest_finished_step[1], available_steps, 
                                                  dependencies, steps_that_follow)
                print("added steps from", quickest_finished_step[1], "these are ", 
                      available_steps)
                priority_queue_time.remove(quickest_finished_step)
                print('removed {} from queue. New queue is {}'.format(quickest_finished_step[1],
                                                                     priority_queue_time))
                print(time_elapsed)
                counter += 1
                
    if priority_queue_time:
        time_elapsed = sorted(priority_queue_time)[0][0]
            
    print('Answer:', time_elapsed)
    return time_elapsed
  

## Test

In [94]:
### Display initial dictionaries with dependencies and steps that follow

dependencies, steps_that_follow = create_dependencies_dictionaries('data/input.txt')

#display dictionary with dependencies
print("Dictionary of Dependencies: ")
for key, value in sorted(dependencies.items()):
    print(key, ": ", value)

#display steps that follow
print("Steps that Follow: ")
for key, value in sorted(steps_that_follow.items()):
    print(key, ": ", value)
    

Dictionary of Dependencies: 
A :  {'U', 'W', 'Y', 'K', 'J', 'L'}
C :  {'H', 'P', 'W', 'Y', 'A', 'I', 'J', 'L'}
D :  {'H', 'Q', 'P', 'N', 'A', 'O', 'J', 'C'}
E :  {'S', 'Q', 'G', 'D', 'A', 'I', 'O', 'J', 'C'}
H :  {'X', 'N', 'I'}
I :  {'X', 'F', 'T'}
J :  {'U', 'Y', 'Z', 'M'}
K :  {'B'}
L :  {'M', 'G', 'K', 'P'}
M :  {'U', 'R', 'K'}
N :  {'F'}
O :  {'U', 'Z', 'Q', 'P', 'V', 'K', 'N', 'L'}
P :  {'G', 'H', 'I'}
Q :  {'U', 'H', 'T', 'W'}
R :  {'B'}
S :  {'X', 'Z', 'Q', 'D', 'Y', 'L', 'N', 'O', 'C'}
T :  {'B'}
U :  {'F', 'T', 'I'}
V :  {'H', 'P', 'W', 'L', 'T'}
W :  {'R', 'T'}
X :  {'R'}
Y :  {'X', 'Q', 'P', 'W', 'B', 'V', 'N', 'I', 'O', 'R'}
Z :  {'B', 'G', 'R', 'T'}
Steps that Follow: 
A :  {'C', 'E', 'D'}
B :  {'Z', 'Y', 'K', 'R', 'T'}
C :  {'E', 'S', 'D'}
D :  {'E', 'S'}
F :  {'U', 'N', 'I'}
G :  {'L', 'Z', 'E', 'P'}
H :  {'Q', 'P', 'D', 'V', 'C'}
I :  {'U', 'H', 'E', 'P', 'Y', 'C'}
J :  {'A', 'C', 'E', 'D'}
K :  {'O', 'A', 'L', 'M'}
L :  {'S', 'V', 'A', 'O', 'C'}
M :  {'J', 'L'}
N :  {

In [95]:
assert main('data/input_test01.txt') == 253

available steps {'C'}
Step to append:  (63, 'C', 0)
priority queue with appended step: [(63, 'C', 0)]
after taking step is worker busy [True, False, False, False, False]
counter for available steps: 0
added steps from C these are  {'A', 'F'}
removed C from queue. New queue is []
63
available steps {'A', 'F'}
Step to append:  (61, 'A', 0)
priority queue with appended step: [(124, 'A', 0)]
after taking step is worker busy [True, False, False, False, False]
available steps {'F'}
Step to append:  (66, 'F', 1)
priority queue with appended step: [(124, 'A', 0), (129, 'F', 1)]
after taking step is worker busy [True, True, False, False, False]
counter for available steps: 0
added steps from A these are  {'B', 'D'}
removed A from queue. New queue is [(129, 'F', 1)]
124
available steps {'B', 'D'}
Step to append:  (62, 'B', 0)
priority queue with appended step: [(129, 'F', 1), (186, 'B', 0)]
after taking step is worker busy [True, True, False, False, False]
available steps {'D'}
Step to append:  

In [96]:
assert main('data/input_test02.txt') == 187

available steps {'A'}
Step to append:  (61, 'A', 0)
priority queue with appended step: [(61, 'A', 0)]
after taking step is worker busy [True, False, False, False, False]
counter for available steps: 0
added steps from A these are  {'C', 'D'}
removed A from queue. New queue is []
61
available steps {'C', 'D'}
Step to append:  (63, 'C', 0)
priority queue with appended step: [(124, 'C', 0)]
after taking step is worker busy [True, False, False, False, False]
available steps {'D'}
Step to append:  (64, 'D', 1)
priority queue with appended step: [(124, 'C', 0), (125, 'D', 1)]
after taking step is worker busy [True, True, False, False, False]
counter for available steps: 0
added steps from C these are  set()
removed C from queue. New queue is [(125, 'D', 1)]
124
counter for available steps: 1
added steps from D these are  {'B'}
removed D from queue. New queue is []
125
available steps {'B'}
Step to append:  (62, 'B', 0)
priority queue with appended step: [(187, 'B', 0)]
after taking step is w

In [92]:
assert main('data/input_test03.txt') == 188

[]
available steps {'C'}
Step to append:  (63, 'C', 0)
priority queue with appended step: [(63, 'C', 0)]
after taking step is worker busy [True, False, False, False, False]
[]
counter for available steps: 0
added steps from C these are  {'D'}
removed C from queue. New queue is []
63
[]
available steps {'D'}
Step to append:  (64, 'D', 0)
priority queue with appended step: [(127, 'D', 0)]
after taking step is worker busy [True, False, False, False, False]
[]
counter for available steps: 0
added steps from D these are  {'A', 'E'}
removed D from queue. New queue is []
127
[]
available steps {'A', 'E'}
Step to append:  (61, 'A', 0)
priority queue with appended step: [(188, 'A', 0)]
after taking step is worker busy [True, False, False, False, False]
[]
available steps {'E'}
Step to append:  (65, 'E', 1)
priority queue with appended step: [(188, 'A', 0), (192, 'E', 1)]
after taking step is worker busy [True, True, False, False, False]
Answer: 188


In [87]:
assert main('data/input_test04.txt') == 337

[]
[]
available steps {'A', 'G', 'F'}
Step to append:  (61, 'A', 0)
priority queue with appended step: [(61, 'A', 0)]
after taking step is worker busy [True, False, False, False, False]
[]
available steps {'G', 'F'}
Step to append:  (66, 'F', 1)
priority queue with appended step: [(61, 'A', 0), (66, 'F', 1)]
after taking step is worker busy [True, True, False, False, False]
[]
available steps {'G'}
Step to append:  (67, 'G', 2)
priority queue with appended step: [(61, 'A', 0), (66, 'F', 1), (67, 'G', 2)]
after taking step is worker busy [True, True, True, False, False]
[]
counter for available steps: 0
added steps from A these are  set()
removed A from queue. New queue is [(66, 'F', 1), (67, 'G', 2)]
61
counter for available steps: 1
added steps from F these are  {'C'}
removed F from queue. New queue is [(67, 'G', 2)]
66
[]
available steps {'C'}
Step to append:  (63, 'C', 0)
priority queue with appended step: [(67, 'G', 2), (129, 'C', 0)]
after taking step is worker busy [True, False, 

<div class="alert alert-success">
<h3>Execute the Program and Solve the Problem!</h3>
</div>

In [44]:
main('data/input.txt')

[]
[]
available steps {'B', 'G', 'F'}
Step to append:  (62, 'B', 0)
priority queue with appended step: [(62, 'B', 0)]
after taking step is worker busy [True, False, False, False, False]
[]
available steps {'G', 'F'}
Step to append:  (66, 'F', 1)
priority queue with appended step: [(62, 'B', 0), (66, 'F', 1)]
after taking step is worker busy [True, True, False, False, False]
[]
available steps {'G'}
Step to append:  (67, 'G', 2)
priority queue with appended step: [(62, 'B', 0), (66, 'F', 1), (67, 'G', 2)]
after taking step is worker busy [True, True, True, False, False]
[]
counter for available steps: 0
added steps from B these are  {'K', 'R', 'T'}
removed B from queue. New queue is [(66, 'F', 1), (67, 'G', 2)]
62
[]
available steps {'K', 'R', 'T'}
Step to append:  (71, 'K', 0)
priority queue with appended step: [(66, 'F', 1), (67, 'G', 2), (133, 'K', 0)]
after taking step is worker busy [True, True, True, False, False]
[]
available steps {'R', 'T'}
Step to append:  (78, 'R', 3)
priorit

<div class="alert alert-info">
<h3>Evaluate & Reflect</h3>
</div>

<div class="alert alert-warning">
  <h5>Evaluate</h5>

  Evaluate your solution (=code) according to the criteria:
  <ol>
    <li>Functionality</li>
    <li>Design & Code</li>
    <li>Readability, Style & Documentation</li>
  </ol>

Write at least one paragraph.

</div>

**Functionality**  

The code solves both part one and part two of the tasks. 
Incremental design shows my progress from buggy non-functional to fully-functional code.

**Design and Code**

The code was broken down into subtasks.  
The data structures were explained.
The reusable and repeatable parts of the code have been packed into functions.

**Readability, Style & Documentation**

I tried to document and explain both the big picture, but also the practical implementation. 
I was more elaborate with this on the first task. The second part had a lot of similarities with the first, so I considered some parts to be already explained and therefore did not go that much into detail. 

In the beginning I sticked to a rather strict doc-string style, which I shifted away from later on, since I found copying the parameter and return type descriptions repetative and rather unnecessary for the purpose of this walkthrough exercise.

<div class="alert alert-warning">
<h5>Reflect</h5>

Reflect on your work. Write at least one paragraph.
</div>

I really enjoyed applying the step-by-step approach, with documentation of bugs, their fixes and my reflection on it. 

Looking through the document, I see how my design has improved and the code became more elegant and understandable. 

Parts to improve: more fine-grained explanation of the design for the second part.